In [1]:
import os
import re
import time
import glob
import numpy as np
import pandas as pd
import lxml
from requests import get
from bs4 import BeautifulSoup
from basketball_reference_scraper.teams import get_roster_stats as grs
from basketball_reference_scraper.utils import remove_accents
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from constants import DATA_DIR #TEAMS, SECONDS_SLEEP

In [2]:
class DataScraper():
    def get_roster_stats1(self, team, season):
        df = grs(team, season)
        return df
    # some players were traded midseason and therefore appear more than once, let's merge them
    def merge_duplicates(self, df):
        duplicate = df.pivot_table(index=['PLAYER'], aggfunc='size')
        duplicate = duplicate[duplicate != 1]
        for player in duplicate.index:
            df1 = df.loc[df['PLAYER'] == player].fillna(0)
            df1 = df1.apply(pd.to_numeric, errors='ignore')
            # create merged row
            merged = df1.iloc[0]
            total_games = df1['G'].sum()
            merged['G'] = total_games
            merged['GS'] = df1['GS'].sum()
            weighted_stats = pd.DataFrame()
            
            for index, row in df1._get_numeric_data().iterrows():
                weight = row['G']/total_games #calculate weight
                new_row = row*weight #muliply weight to get new values
                weighted_stats = weighted_stats.append(new_row) #add to df
            
            weighted_stats = weighted_stats.sum(axis=0, skipna=True)
            merged_stats = ['MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA', 
                            'ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
            
            for stat in merged_stats:
                merged[stat] = weighted_stats[stat]
            
            # calulate % stats
            merged['FG%'] = merged['FG']/merged['FGA']
            merged['3P%'] = merged['3P']/merged['3PA']
            merged['2P%'] = merged['2P']/merged['2PA']
            merged['eFG%'] = (merged['FG']+0.5*merged['3P'])/merged['FGA']
            merged['FT%'] = merged['FT']/merged['FTA']
            merged=merged.fillna(0)
            
            df = df[df.PLAYER != player]
            df = df.append(merged)
           
        return df
        

In [3]:
if not os.path.exists(os.path.join(DATA_DIR,'PlayerStats')):
    os.mkdir(os.path.join(DATA_DIR,'PlayerStats'))

SEASONS = [2017,2018,2019,2020,2021]
TEAMS  = ['ATL','BRK','BOS','CHI','CHO','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM',
          'MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS']

scraper = DataScraper()
for season in SEASONS:
    df = pd.DataFrame()
    for team in TEAMS:
        print(team)
        df1 = scraper.get_roster_stats1(team, season)
        df = df.append(df1)
    df.to_csv('test.csv', index=False)
    df = scraper.merge_duplicates(df)    
    df.to_csv(DATA_DIR+'/PlayerStats/'+f'{season-1}-{str(season)[2:]}.csv', index=False)

ATL
BRK
BOS
CHI
CHO
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS


/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
<ipython-input-2-a556c8b1f233>:33: RuntimeWarning: invalid value encountered in double_scalars
  merged['3P%'] = merged['3P']/merged['3PA']
<ipython-input-2-a556c8b1f233>:36: RuntimeWarning: invalid value encountered in double_scalars
  merged['FT%'] = merged['FT']/merged['FTA']


ATL
BRK
BOS
CHI
CHO
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS


<ipython-input-2-a556c8b1f233>:34: RuntimeWarning: invalid value encountered in double_scalars
  merged['2P%'] = merged['2P']/merged['2PA']


ATL
BRK
BOS
CHI
CHO
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHI
CHO
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHI
CHO
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
